# 1.从牌谱文件中还原对局

#### 1.1 读取牌谱文件中的洗牌信息，并根据\<init\>标签初始化table

In [1]:
from pymahjong.replayer import myReplayer
from base_module import *
import os
from multi_extract_data import *
from multiprocessing import Queue, Process, current_process
from pymahjong.myEnv_pymahjong import myMahjongEnv

year = 2015
path = f'/data/satori_hdd4/Ren/dataset/{year}'
file_name_list = os.listdir(path)




In [36]:
def replay(path, file_name):
    replayer = myReplayer(path, [file_name])
    # for file_name in file_name_list:
    if True:
        tree = replayer._open_file(path=path, file_name=file_name)
        return_data = replayer._get_inst_and_action_list(tree)
        if return_data is None:
            return None


        # for kyoku in range(len(return_data['replayer'])):
        if True:
            kyoku = 0
            queue = return_data['action_list'][kyoku].copy()
            env = myMahjongEnv()
            obs, info = env.reset(table=return_data['replayer'][kyoku].table)

            done = False
            while not done:
                if queue[1] == 12 + 47 * 3:
                    return env, queue

                is_riichi = [player.riichi for player in env.t.players]
                if len(queue) > 0:

                    while is_riichi[queue[0]//47] and queue[0]%47 < 34:
                        # print(f'player {queue[0]//47} is riichi')
                        # print(f'skip {reversed_encoding[queue[0]]}')
                        queue.popleft()

                    action = queue[0]
                curr_player = env.get_curr_player_id()
                legal_actions = env.get_valid_actions()
                action_idx = action % 47
                player_idx = action // 47
                if action_idx == 44: # 九种九牌
                    player_idx = curr_player
                if curr_player == player_idx and action_idx in legal_actions:
                    obs, reward, done, _, info = env.step(action_idx=action_idx)
                    # print(f'{reversed_encoding[action]}')
                    if len(queue) > 0:
                        queue.popleft()
                    else:
                        print(f'file_name {file_name} kyoku {kyoku} failed')
                        print(f'player {player_idx} action {reversed_encoding[action]} is invalid')
                        print(f'curr_player {curr_player} legal_actions {legal_actions}')
                        return env, return_data['action_list'][kyoku]
                elif 45 in legal_actions:
                    obs, reward, done, _, info = env.step(action_idx=45)
                    # print(f'player {curr_player} pass_naki')

                elif 46 in legal_actions:
                    obs, reward, done, _, info = env.step(action_idx=46)
                    # print(f'player {curr_player} pass_riichi')
                    
                else:
                    print(f'file_name {file_name} kyoku {kyoku} failed')
                    print(f'player {player_idx} action {reversed_encoding[action]} is invalid')
                    print(f'curr_player {curr_player} legal_actions {legal_actions}')
                    return env, return_data['action_list'][kyoku]
                    
            # print(f'file_name {file_name} kyoku {kyoku} success')
    return info['obs_with_return']


In [ ]:
queue = 

In [37]:
env, queue = replay(path, '15641.gz')

In [38]:
for a in queue:
    print(reversed_encoding[a])

('player2', 'discard', '6z')
('player3', 'discard', '4p')
('player0', 'discard', '8s')
('player3', 'ron')


In [45]:
print(env.get_curr_player_id())
print(env.get_valid_actions())

0
[35, 45]


In [46]:
print(env.t.to_string())

Yama: 2s 2m 6z 1p 4z 0p 6z 0s 6m 8m 3z 3s 1m 5z (Wanpai)| 3s 1s 7s 8s 6m 3m 7m 4s 9s 1p 2p 9s 4s 7m 8m 2z 3m 6m 8m 5s 3p 3z 9p 6p 9m 2m 5z 8p 7z 5m 3z 4z 4m 3m 2m 8p 5p 5p 2p 9p 5m 4z 7p 8s 
Dora Indicator(s):4z 6z 
Remaining tiles: 44
Player 0: 
Pt: 25000
Wind: East
Hand: 6p 7p 8p 2s 3s 4s 5s 1z 1z 1z 
Calls: (7m)8m9m 
River: 1m1h 1m5 7z9 5z13h 1p17h 4m21h 6z25h 
Riichi: No
Menzen: No
Player 1: 
Pt: 25000
Wind: South
Hand: 0m 5m 6m 7m 9m 3p 8p 9p 1s 2s 5s 6s 7s 
River: 9s2h- 2z6h 4z10 2m14h 3s18h- 2z22h 1s26h 
Riichi: No
Menzen: Yes
Player 2: 
Pt: 25000
Wind: West
Hand: 3m 2p 6s 6s 6s 8s 3z 
Calls: 7s8s(9s) 2s(3s)4s 
River: 3p3h 6p7h 9m11 9p15 2z19h 5z23h 6z27h 
Riichi: No
Menzen: No
Player 3: 
Pt: 24000
Wind: North
Hand: 4m 4m 1p 2p 3p 5p 6p 7p 7s 9s 
Calls: 4p**4p 
River: 1m4h 7z8h 7m12h- 1z16h 7z20r 1s24r 7p28r 
Riichi: Yes
Menzen: Yes

Oya player 0
Honba: 0
Kyoutaku: 1



In [44]:
env.step(38)

({'tiles_features': array([[[False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False,  True,  True,  True, False],
          [False,  True,  True,  True,  True, False, False, False, False],
          [ True, False, False, False, False, False, False, False, False]],
  
         [[False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False],
          [ True, False, False, False, False, False, False, False, False]],
  
         [[False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False],
          [ True, False, False, False, False, False, False, False, False]],
  
         [[False, False, False, False, False, False, False, False, Fal

## 多线程

In [3]:
from multiprocessing import Queue, Process, current_process

def worker(input, output):
    for func, args in iter(input.get, 'STOP'):
        result = func(*args)
        # if result is not None:
        output.put(result)

def calculate(func, args):
    result = func(*args)
    return result

In [10]:
input = Queue()
output = Queue()
num_worker = 64

dataset = []
for file_name in file_name_list:
    input.put((replay, (path, file_name)))

for i in range(num_worker):
    Process(target=worker, args=(input, output)).start()

for i in range(len(file_name_list[:1000])):
    data = output.get()
    if data is not None:
        dataset.extend(data)

for i in range(num_worker):
    input.put('STOP')
    


file_name 6595.gz kyoku 0 failed
player 0 action ('player0', 'ron') is invalid
curr_player 0 legal_actions [42, 45]


file_name 15641.gz kyoku 0 failed
player 0 action ('player0', 'discard', '8s') is invalid
curr_player 3 legal_actions [12, 38]
file_name 10370.gz kyoku 0 failed
player 3 action ('player3', 'discard', '4z') is invalid
curr_player 3 legal_actions [0, 2, 3, 7, 9, 11, 13, 15, 17, 21]
file_name 12855.gz kyoku 0 failed
player 0 action ('player0', 'tsumo') is invalid
curr_player 0 legal_actions [1, 2, 3, 5, 7, 12, 13, 14, 22]
file_name 264.gz kyoku 0 failed
player 2 action ('player2', 'discard', '1s') is invalid
curr_player 3 legal_actions [2, 3, 4, 11, 12, 13, 20, 21, 22, 23, 28]
file_name 1611.gz kyoku 0 failed
player 1 action ('player1', 'discard', '5s') is invalid
curr_player 0 legal_actions [38, 22]
file_name 11606.gz kyoku 0 failed
player 0 action ('player0', 'ron') is invalid
curr_player 0 legal_actions [42, 45]
file_name 13427.gz kyoku 0 failed
player 3 action ('player3', 'discard', '3m') is invalid
curr_player 3 legal_actions [32, 4, 12, 13, 14, 27, 31]
file_name 15854.gz kyoku 0 fai

In [21]:
while output.qsize() > 0:
    data = output.get()
    if data is not None:
        dataset.extend(data)

Process Process-200:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_490693/3055941648.py", line 5, in worker
    result = func(*args)
  File "/tmp/ipykernel_490693/3583977432.py", line 26, in replay
    while is_riichi[queue[0]//47] and queue[0]%47 < 34:
IndexError: deque index out of range
Process Process-228:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_490693/3055941648.py", line 5, in worker
    result = func(*args)
  File "/tmp/ipykernel_490693/3583977432.py", line 26, in replay
    while is_riichi[queue[0]//47] and queue[0]%47 < 34:

In [22]:
len(dataset)

93444

In [ ]:
print(env.get_curr_player_id())
print(env.get_valid_actions())

In [ ]:
print(env.t.to_string())

In [ ]:
# import torch

# kyoku = 0
# player = 0

# training_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(training_device)

# config = GPT2Config(n_embd=512, n_layer=8, n_head=8, n_positions=128)
# model = Policy_Network(config).to(training_device)

# input_collator = myCollator(device=training_device)
# input = []
# labels = []
# for kyoku, label in zip(data['obs'], data['label']):
#     input.extend(kyoku)
#     labels.extend(label)
# input = input_collator(input)
# output = model(**input)


# label_collator = label_Collator(device=training_device)
# label = label_collator(labels)

# criteria = torch.nn.CrossEntropyLoss()
# loss = criteria(output['action_logits'], label)
# print(input['action_list'].shape)
# print(label.shape)
# print(loss)